# Implementação de Métodos de Machine Learning
**Aluno:** Matheus Gama dos Santos - 20180163117<br>
**Curso:** Ciência da Computação - UFPB<br>
**Profª:** Thais Gaudencio do Rego<br><br>
Este projeto consiste na implementação de métodos de Machine Learning para analisar as bases de dados listadas abaixo:
- [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/data)
- [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)
- [2018-2019 Premier League Data
](https://www.kaggle.com/thesiff/premierleague1819)

Cada base de dados será tratada em uma das 3 seções: **Classificação**, **Regressão** e **Clusterização** (nessa ordem).<br>

## Classificação

In [ ]:
#install imblearn
!pip install imblearn

In [ ]:
#import modules
import pandas as pd
import numpy as np
from sklearn import *
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.metrics import sensitivity_score, specificity_score
from string import ascii_letters
import ipywidgets as widgets

In [ ]:
#load dataframe
titanic_filename = 'titanic/train.csv'
titanic_dataframe = pd.read_csv(titanic_filename)

In [ ]:
titanic_dataframe.head()

### Preprocessing

In [ ]:
# select useful attributes
titanic_dataframe = titanic_dataframe.drop(["PassengerId", "Name", "Ticket", "Cabin", "Embarked", "Fare"], axis=1)
titanic_dataframe.head()

In [ ]:
# converting "Sex" attribute
titanic_dataframe.Sex = titanic_dataframe.Sex.replace({'male':0, 'female':1})
titanic_dataframe.head()

In [ ]:
titanic_dataframe.describe()

<font size="3">&emsp;&emsp;Analisando o estado atual do *Dataframe*, observa-se que o atributo *Age* possui mais de 100 linhas sem dado. Vamos realizar testes para verificar a influência das linhas com dado faltando.</font><br><br>

In [ ]:
def get_titanic_survivability(d_frame_surv): # d_frame_surv: dataframe Survived column
    zeros = 0
    ones = 0
    for numbers in d_frame_surv:
        if numbers:
            ones += 1
        else:
            zeros += 1
    print(f"Died: {zeros} ({zeros/(zeros+ones)}), Survived: {ones} ({ones/(zeros+ones)})")

In [ ]:
# Compare dataframe with and without missing data rows

# m_df will be titanic_dataframe.Survived without missing values
m_df = titanic_dataframe.dropna(axis=0)
m_df = np.array(m_df.Survived)

# dtf will be a copy of titanic_dataframe.Survived
df = np.array(titanic_dataframe.Survived)

In [ ]:
# Compare results
print('Without missing values:')
get_titanic_survivability(m_df)
print('\nWith missing values:')
get_titanic_survivability(df)

<font size="3">&emsp;&emsp;É possível observar que o balanceamento da base de dados, com relação ao *target* ( *Survived* ), não teve alteração significativa ao eliminar os objetos sem o valor do atributo *Age*. Vamos comparar, por fim, o impacto da mudança nos outros atributos.</font><br><br>

In [ ]:
# description of titanic_dataframe
titanic_dataframe.describe()

In [ ]:
# description of titanic_dataframe without the missing values
titanic_dataframe.dropna(axis=0).describe()

<font size="3">&emsp;&emsp;O impacto não se mostrou significativo na distribuição dos outros atributos. Agora, vamos normalizar os dados e exibir a matriz de correlação dos atributos relevantes.</font><br><br>

In [ ]:
titanic_dataframe = titanic_dataframe.dropna(axis=0)

# Normalizing dataframe
def normalize_df(df):
    d_values = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    d_values_scaled = min_max_scaler.fit_transform(d_values)
    df_norm = pd.DataFrame(d_values_scaled, columns=df.columns)
    return df_norm

titanic_dataframe_norm = normalize_df(titanic_dataframe)
titanic_dataframe_norm.sample(5)

In [ ]:
# Correlation Matrix
sns.heatmap(titanic_dataframe_norm.corr(), annot=True, fmt=".2f")
plt.show()

<font size="3">&emsp;&emsp;Em relação ao *target* (*Survived* ), é possível perceber que as correlações mais importantes são entre a classe do indivíduo ( -0,36 ) e o sexo ( 0,54 ). Quanto menor a classe social, mais chance de sobreviver. Também pode-se dizer que o sexo feminino tem mais chance de sobreviver do que o masculino. As outras correlações não apresentam valores expressivos. Analisaremos agora a presença de outliers em cada atributo.</font><br><br>

In [ ]:
titanic_dataframe.describe()

<font size="3">&emsp;&emsp;*Survived* é um atributo binário, que também é o target, portanto seus valores tem significado classificatório. Logo, survived não possui outliers. Pclass também possui valores com significado classificatório (1, 2 ou 3), logo não possui outliers. Sex é binário e também possui significado classificatório, logo não possui outliers. Age, aparentemente possui outliers, dado que sua média tem valor aproximado de 29,6991, com desvio padrão de aproximadamente 14,5264. Se considerarmos ( $média + (2dp)$ ), valores acima de 58,7519 podem ser considerados outliers. Porém, nesse caso, a base de dados possui valores entre 58,7519 e 80, que são valores representativos pra base de dados. Além do mais, talvez se utilizássemos ( $média + (3dp)$ ) não consideraríamos como outlier. O mesmo acontece com SibSp e Parch. Para verificar as afirmações, vamos observar os gráficos de cada atributo.</font><br><br>

In [ ]:
attribute = 'Survived'
def on_change(change):
    global attribute
    if change['type'] == 'change' and change['name'] == 'value':
        attribute = change['new']

In [ ]:
w = widgets.Dropdown(
    options=[e for e in titanic_dataframe.columns],
    value=titanic_dataframe.columns[0],
    description='Attribute:',
    disabled=False,
)

w.observe(on_change)

display(w)

##### Obs.: Selecione o atributo e execute as duas próximas células para exibir o histograma e boxplot

In [ ]:
# Plot histogram
titanic_dataframe[attribute].plot(kind='hist',title=attribute ,bins=50,figsize=(8,4))

In [ ]:
# Boxplot
titanic_dataframe[attribute].plot(kind='box',figsize=(8,4))

<font size="3">&emsp;&emsp;Ao observar os gráficos, vemos que SibSp e Parch são os atributos com verdadeiros outliers e não balanceados por essa mesma razão. Vamos retirar então os valores acima de ( $média + ( 3dp )$ ) dos dois atributos.</font><br><br>

In [ ]:
# Dropping outliers
indexNames = titanic_dataframe[ (titanic_dataframe['SibSp'] > 3) | (titanic_dataframe['Parch'] > 3) ].index
titanic_dataframe.drop(indexNames , inplace=True)

# get normalized version
titanic_dataframe_norm = normalize_df(titanic_dataframe)

<font size="3">Agora temos os dados balanceados e normalizados, prontos para aplicar os modelos.</font><br><br>

### Models

In [ ]:
# select features and target
y = titanic_dataframe_norm.Survived
x = titanic_dataframe_norm.drop(columns='Survived').to_numpy()

In [ ]:
# split train and test
train_X, test_X, train_Y, test_Y = model_selection.train_test_split( x, y, random_state=0, test_size=.2 )

<font size="3">&emsp;&emsp;Os modelos escolhidos foram Random Forest e SVM, com a razão da escolha de cada um descrita junto com os resultados após a execução.</font><br><br>

In [ ]:
# Random Forest
rdf = ensemble.RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2) #max_depth = 3, random_state=2
rdf.fit(train_X, train_Y)

In [ ]:
pred_Y = rdf.predict(test_X)
def show_metrics(test, pred):
    # accuracy: the fraction of predictions the model got right.
    accuracy = metrics.accuracy_score(test,pred)

    # sensitivy (recall): Percentage of correct predictions out of all the positive classes. (TP / TP+FN).
    sensitivity = sensitivity_score(test,pred)
    
    # specificity: proportion of actual negatives that are correctly identified as such. (TN / FP + TN).
    specificity = specificity_score(test,pred)
    
    """>Precision (used to calculate f1 score): Out of all the positive classes predicted correctly, how many are actually positive. (TP / TP+FP)"""
    # f1 score (harmonic mean of precision and recall): an accuracy coefficient with values between 0 and 1 (0 and 1 included).
    f1_score = metrics.f1_score(test, pred)

    x = np.array([accuracy, f1_score, sensitivity, specificity]).reshape(1,4)
    return pd.DataFrame(x, columns=["Accuracy", "F1 Score","Sensitivity", "Specificity"], index=["Results"])

results_r_forest = show_metrics(test_Y, pred_Y)
results_r_forest

In [ ]:
# confusion matrix
def show_confusion_mat(test,pred):
    c_mat = metrics.confusion_matrix(test, pred)
    return pd.DataFrame(c_mat, columns=["P", "N"], index=["P", "N"])
mat_r_forest = show_confusion_mat(test_Y,pred_Y)
mat_r_forest

# TP  FP
# FN  TN

<font size="3">&emsp;&emsp;Random Forest é um algoritmo de classificação e regressão, baseado em árvores de decisão, porém seu funcionamento corrige o hábito de realizar *overfitting* sobre a base de treinamento. Com acurácia de 0,861314 e F1-Score de 0,828829, o modelo se mostrou bem eficiente. Utilizando *max_depth* = 3, *random_state* = 2, obteve-se sensibilidade de 0,793103 e especificidade de 0,911392, que podem ser considerados bons resultados. Os valores para os parâmetros utilizados foram obtidos através de testes manuais e comparação.</font><br><br>

In [ ]:
# SVC (SVM)
SVC = svm.SVC(gamma=11)
SVC.fit(train_X, train_Y)

In [ ]:
pred_Y = SVC.predict(test_X)
results_SVC = show_metrics(test_Y, pred_Y)
results_SVC

In [ ]:
mat_SVC = show_confusion_mat(test_Y,pred_Y)
mat_SVC

<font size="3">&emsp;&emsp;SVM é classificador linear binário não probabilístico que tenta definir uma reta que melhor separa as classes do problema. O fato da razão entre as saídas ser próximo de 1 e os objetos da base só possuírem apenas duas classificações possíveis são a justificativa para a escolha desse algoritmo. Os resultados foram bem satisfatórios, com acurácia de 0,854015 e F1-Score de 0,821429. Os números de acertos positivos e negativos também encontrou altas taxas, com sensibilidade de 0,793103 e especificidade de 0,898734. Foram testados alguns valores para o parâmetro *gamma*, que foi o mais impactante nos resultados quando alterado, dentro do intervalo de 1 a 15, com melhor valor sendo *gamma* = 11. A mudança do gamma para outros valores ocasiona em troca de acurácia e especificidade por sensibilidade, ou apenas a perda em algum dos valores. O estado atual mantém sensibilidade e especificidade balanceados, diminuindo a chance de *overfitting*.</font><br><br>

## Regression

In [ ]:
#load dataframes
hp_filename = 'house_prices/train.csv'
hp_dataframe = pd.read_csv(hp_filename, na_values=None,keep_default_na=False)

In [ ]:
hp_dataframe.describe()

### Preprocessing

In [ ]:
# converting non-numerical attributes
file = open('house_prices/dict.txt','r')
replaces = file.read()
replaces = replaces.split(";")
for each_element in replaces:
    attribute = each_element[2:each_element.find("'",2)]
    dictionary = each_element[each_element.find("{",2) +1: each_element.find("}")].split(', ')
    dict_rep = {}
    for each_n_element in dictionary:
        value = each_n_element[each_n_element.find(": ")+2:]
        dict_rep[each_n_element[1:each_n_element.find("'",2)]] = int(value)
    hp_dataframe[attribute].replace(dict_rep, inplace=True)
file.close()
hp_dataframe[hp_dataframe.select_dtypes("object").columns] = hp_dataframe[hp_dataframe.select_dtypes("object").columns].astype('int', inplace=True)

In [ ]:
# normalizing dataframe
hp_dataframe = normalize_df(hp_dataframe)
hp_dataframe.drop(columns=['Id'], inplace=True)
hp_dataframe.sample(5)

### Models

In [ ]:
# select features and target
y = hp_dataframe.SalePrice
x = hp_dataframe.drop(columns='SalePrice').to_numpy()
# split train and test
train_X, test_X, train_Y, test_Y = model_selection.train_test_split( x, y, random_state=0, test_size=.2 )

<font size="3">&emsp;&emsp;Os modelos escolhidos foram *Linear Regression* e *Random Forest Regression*. *Linear Regression* foi escolhido com o pensamento de que cada um dos atributos é indepente entre si, mas compõem a saída em conjunto. O motivo da escolha do *Random Forest* foi baseada na ideia de que *Decision Trees* tentam tomar decisões seguindo caminhos que melhor representem a situação, sendo a ideia parecida com o problema adotado (classificar o preço de uma casa baseado no preço de outras). *Random Forest* usa várias árvores de decisão e escolher a que melhor representa a ocasião, também com a vantagem de corrigir o hábito de *overfitting*, como já citado.</font><br><br>

In [ ]:
# Linear Regression
lr = linear_model.LinearRegression().fit(train_X,train_Y)

In [ ]:
lr_pred = lr.predict(test_X)
# Erro Médio Absoluto
metrics.mean_absolute_error(lr_pred,test_Y)

In [ ]:
# Erro Quadrático Médio
metrics.mean_squared_error(lr_pred,test_Y)

<font size="3">&emsp;&emsp;Comment</font><br><br>

In [ ]:
# Random Forest
rf_reg = ensemble.RandomForestRegressor(n_estimators=400, max_depth=25, random_state=1).fit(train_X,train_Y)

In [ ]:
rf_pred = rf_reg.predict(test_X)
# Erro Médio Absoluto
metrics.mean_absolute_error(rf_pred,test_Y)

In [ ]:
metrics.mean_squared_error(rf_pred,test_Y)

<font size="3">&emsp;&emsp;Comment</font><br><br>